Adapted from [ner_with_bilstm_and_crf](https://www.kaggle.com/nikkisharma536/ner-with-bilstm-and-crf/notebook)
Altigran Soares da Silva
IComp/UFAM - 15/03/2021


In [1]:
!pip install chemdataextractor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 4.1 MB/s 
     |████████████████████████████████| 5.6 MB 47.8 MB/s 
     |████████████████████████████████| 371 kB 49.6 MB/s 
     |████████████████████████████████| 965 kB 59.6 MB/s 
     |████████████████████████████████| 4.1 MB 43.1 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=863525 sha256=2ffed5eaf5fbb502be6a0785fbec6bf9ee987cd1275805ff00aee9877e5fdf68
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [2]:
!cde data download

INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/cem_crf-1.0.pickle to /root/.local/share/ChemDataExtractor/models/cem_crf-1.0.pickle
INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/cem_crf_chemdner_cemp-1.0.pickle to /root/.local/share/ChemDataExtractor/models/cem_crf_chemdner_cemp-1.0.pickle
INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/cem_dict_cs-1.0.pickle to /root/.local/share/ChemDataExtractor/models/cem_dict_cs-1.0.pickle
INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/cem_dict-1.0.pickle to /root/.local/share/ChemDataExtractor/models/cem_dict-1.0.pickle
INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/clusters_chem1500-1.0.pickle to /root/.local/share/ChemDataExtractor/models/clusters_chem1500-1.0.pickle
INFO:chemdataextractor.data:Downloading http://data.chemdataextractor.org/models/pos_ap_genia_nocluster-1.0.pick

In [3]:
from chemdataextractor.doc import Paragraph
para = Paragraph('1,4-Dibromoanthracene was prepared from 1,4-diaminoanthraquinone. 1H NMR spectra were recorded on a 300 MHz BRUKER DPX300 spectrometer.')
para.sentences

[Sentence('1,4-Dibromoanthracene was prepared from 1,4-diaminoanthraquinone.', 0, 65),
 Sentence('1H NMR spectra were recorded on a 300 MHz BRUKER DPX300 spectrometer.', 66, 135)]

In [25]:
from os import listdir
from chemdataextractor.nlp.tokenize import ChemWordTokenizer

files=[
'101016jelecom200809031',
'101016jjssc200512036',
'101016jcattod201404032',
'101002pssa200881782',
'101039c7ta03244g',
'101002adma201300071',
'101016jscriptamat201603010',
'101016jjpowsour201403088',
'101039c6ta03162e',
'101039c4ta03633f',
'101016jelectacta201410035',
'101016jjpowsour201207060',
'101039c4ta03692a',
'101016jelecom201507001',
'101039c4cy00238e',
'101039c6ta09601h',
'101039b512481f',
'101016jelectacta201605074',
'101016jssc200602027',
'101038s41598-017-15535-x',
'101016jjallcom201511182',
'101016jjallcom201503081',
'101016jcorsci201601031',
'101039c6ta09941f',
'101016jpolymer201604016',
'101038ncomms3365',
'101016jjcat201305016',
'101016jjelechem201503033',
'101016jelectacta201504007',
'101016jelectacta201404084',
'101016jelectacta201201094',
'101039c5ta02179k',
'101016jmseb201512003',
'101016jelectacta201504160',
'101016jelectacta201303189',
'101016jtsf201301068',
'101002cctc201200482',
'101016jcarbon201403017',
'101016jjpowsour201603101',
'101016jmaterresbull201504049',
'101039c6ta05384j',
'101016jtsf201309083',
'101039c6ta02139e',
'101016jmatlet201312007',
'101039c5ce02131f',
'101016jphyse201309020',
'101016jijhydene200912145',
'101016jssi201505024',
'101039c5ta03331d',
'101016jelecom200308018',
'101016jelectacta201002016',
'101016jmatlet201301055',
'101016jbios201407031',
'101016jmatlet201305115',
'101016jcolsurfa201208033',
'101016jjallcom201209067',
'101002cssc201200775',
'101016jjpowsour201202061',
'101016jjmmm201211056',
'101002cplu201402021',
'101016jssi200406013',
'101016jmaterresbull201102029',
'101039b910641c',
'101038srep35549',
'101039c5nr00656b',
'101016jenconman201111001',
'101016jelectacta201510124',
'101039c3ee41655k',
'101016jconbuildmat201510097',
'101016jmolcata200404032',
'101038srep45030',
'101016jjpowsour200610051',
'101016jjcis201505009',
'101016jjcis201501016',
'101039c7ta05788a',
'101016jmolcata200510024',
'101016jconbuildmat201308047',
'101016jjpowsour201401073',
'101016jjpowsour201601014',
'101016jcolsurfa201501068',
'101016jmatdes201608043',
'101016jjpowsour201110077',
'101016jspmi201405032',
'101016jssi201001008',
'101002aoc1683',
'101016japcata200605025',
'101039c6nr03086f',
'101016jelectacta201309101',
'101016jmssp201402048',
'101016jmaterresbull200706005',
'101039c7nr05695h',
'101007s11581-014-1364-0',
'101016jmee201501022',
'101039c5cy00124b',
'101016jmatlet201110029',
'101016jmatlet201001077',
'101039c5ta08193a',
'101016jjpowsour201412144',
'101039c5cy01175b',
'101016jmatdes201604043',
'101016jmatlet201405013',
'101016jelectacta201409032',
'101039c6cy00580b',
'101002rcm7385',
'101016jelectacta201309128',
'101016jelectacta201604152',
'101039c7ta03087h',
'101038srep14146',
'101039c5cy02011e',
'101039c5cy01252j',
'101016jjeurceramsoc201504038',
'101016jpowtec201311008',
'101016jelectacta201512002',
'101016jjpowsour201507066',
'101016jelectacta201501051',
'101016jtetlet201004103',
'101016jmatlet201112019',
'101038ncomms5358',
'101016jjcat201212002',
'101002pssb201200101',
'101016jtsf201212064',
'101039c5ta03160e',
'101002pssa201127501',
'101016jjssc200402014',
'101016jtet201005091',
'101016jijhydene201105188',
'101016jelecom201701017',
'101002adsc201100908',
'101016jjpowsour201203049',
'101038srep36857',
'101016jjcis201408027',
'101002adma200903953',
'101016jjallcom201412037',
'101016jmatlet201211079',
'101016jnanoen201603001',
'101016jjelechem201405031',
'101016jmatlet200508058',
'101016jelectacta201611160',
'101039c5ta05691h',
'101016japcatb201305035',
'101016jelectacta201212112',
'101016jsnb201503104',
'101016jsolmat201308023',
'101038srep23400',
'101016jjssc201203030',
'101016jssi201005047',
'101016jnanoen201507001',
'101016jsolmat201109031',
'101016jjpowsour201110028',
'101039c6ta02230h',
'101016jcplett201608020',
'101039c4ta05178e',
'101016jsolidstatesciences200803021',
'101016jjcis201406018',
'101016jsolener201610023',
'101016jmssp201501013',
'101016jelectacta201502218',
'101016jelectacta201504084',
'101016jjpowsour201207012',
'101016jcej201005016',
'101016jcplett201504042',
'101039c4ce01493f',
'101016jelectacta201008097',
'101038s41598-017-16585-x',
'101002aenm201100654',
'101016jmatlet201403043',
'101016jnanoen201502025',
'101016jsusc200507041',
'101016jjpowsour201601010',
'101103PhysRevB81205204',
'101016jcattod201304013',
'101016jelectacta201510036',
'101016jmatchemphys200710005',
'101002ejic201201429',
'101016jjallcom201312206',
'101016jconbuildmat201405031',
'101016jjpowsour201006097',
'101038srep39770',
'101007s11164-011-0277-z',
'101039c3dt53493f',
'101016jjpowsour201603050',
'101039c5gc00379b',
'101016jcej201504032',
'101039c4ta04367g',
'101039c6ce01349j',
'101016jjpowsour201205003',
'101039c5ta03565a',
'101016jpowtec201511012',
'101016jelectacta201210092',
'101016jjcat201301010',
'101016jjpowsour201501158',
'101021nn9002456',
'101002app40112',
'101103PhysRevB96054448',
'101016jjallcom201406086',
'101039c5tc04364f',
'101016jjpowsour201204074',
'101002app29929',
'101002app20681',
'101039c5nr00246j',
]
ann_files= [f for f in listdir('test') if f.endswith(".ann") and f[:-4] in files]
cwt = ChemWordTokenizer()

# print(ann_files)

count = 1
result = []

testeeee = 0
testeeee2 = 0

# ann_files = ['101016jjpowsour201203049.ann']

for ann_file in ann_files:
    txt_file = f"{ann_file.split('.ann')[0]}.txt"
    print(f"Processing files: {ann_file}, {txt_file}")

    f1 = open(f"test/{ann_file}", "r", newline="\n", encoding="utf-8")
    f2 = open(f"test/{txt_file}", "r", newline="\n", encoding="utf-8")

    tags = []

    f1_lines = f1.readlines()
    for line in f1_lines:
        columns = line.split("\t")
        if(len(columns) > 3):
            raise "ERROR1"
        if(columns[0].startswith("T")):
            # print(columns)

            tag = columns[1].split()
            # print(tag)
            if(len(tag) > 3):
                raise "ERROR2"

            # tags.append(tag[0])
            # tags_indexes.append((int(tag[1]), int(tag[2])))
    
            # words.append([word for word in columns[2].replace("\n", "").split()])

            tags.append({
                'tag': f"{tag[0]}",
                'tags_indexes': (int(tag[1]), int(tag[2])),
                'word': [word for word in columns[2].replace("\n", "").split()]
            })
            # print(words)

    print(tags)
    print(len(tags))

    from chemdataextractor.doc import Paragraph
    # from nltk.tokenize import TreebankWordTokenizer as twt

    f2_lines = f2.readlines()
    print(f2_lines)
    # print(nltk.sent_tokenize(f2_lines[1]))
    # testeeee = testeeee + len(f2_lines)
    teste_sentences = [[str(sent) for sent in Paragraph(line).sentences] for line in f2_lines]
    print(teste_sentences)
    teste_sentences = [line if len(line)>0 else ' ' for line in teste_sentences]
    print(teste_sentences)
    teste_sentences = [i for j in teste_sentences for i in j]
    # print(teste_sentences)
    # teste_sentences = [str(sent) for sent in teste_sentences]
    print(teste_sentences)
    # testeeee2 = testeeee2 + sum([len(line) for line in teste_sentences])
    testeeee2 = testeeee2 + len(teste_sentences)
    # print(teste_sentences)
    teste_sentences = [line if len(line)>0 else ' ' for line in teste_sentences]
    print(teste_sentences)
    # teste_sentences = [i for j in teste_sentences for i in j]
    # print(teste_sentences)
    # print([list(cwt.span_tokenize(line)) for line in f2_lines])
    # print(len(f2_lines))
    line_lengths = [list(cwt.span_tokenize(line))[-1][1] if len(list(cwt.span_tokenize(line))) else 0 for line in teste_sentences]
    line_end_indexes = [ i+sum(line_lengths[0:k])+k if k!=0 else i for k,i in enumerate(line_lengths)]

    fixes = 0
    full_text = "".join(f2_lines) + "\n"
    print(len(full_text))
    for i in range(len(line_end_indexes)):
      print(i)
      print(line_end_indexes)
      line_end_indexes[i] = line_end_indexes[i]-fixes
      if not full_text[line_end_indexes[i]].isspace():
        fixes = fixes + 1

    erase_lines = [k for k, line in enumerate(line_lengths) if line==0]
    print(erase_lines)
    line_end_indexes = [line for k, line in enumerate(line_end_indexes) if k not in erase_lines]
    print(line_end_indexes)
    # print(line_end_indexes)
    # teste=" ".join(line.strip() for line in teste_sentences) + "\n"
    # print(teste)
    # print(teste.find("synthesized"))
    # print(teste.find("critic acid"))
    # print(teste.find("ethanol"))
    # print(teste.find("argon"))

    # from nltk.tokenize import word_tokenize, span_tokenize
    # nltk.download('punkt')
    # print(word_tokenize("Hello, my name is Arthur. How are you?"))
    teste2 = list(cwt.span_tokenize(full_text))
    # print(teste)
    # print(len(teste))
    # print(teste2)

    # teste3 = list(twt().tokenize(teste))
    # print(teste3)
    # teste4 = list(twt().tokenize("Hello, my name is Arthur. How are you?"))
    # print(teste4)

    # print([teste[word_index[0]:word_index[1]] for word_index in teste2])

    line_counter = 0

    # print(tags)

    # print(teste2)
    for word_index in teste2:
        # print(line_counter, line_end_indexes, word_index[0])
        if word_index[0] > line_end_indexes[line_counter]:
            count = count+1
            line_counter = line_counter+1

        word = full_text[word_index[0]:word_index[1]]
        if('"' in word):
            word = word.replace('"', "'")
        if("," in word or "=" in word):
            word = f'"{word}"'

        # print(word)
        # print(word_index[0],word_index[1])
        
        found = False
        for tag in tags:
            if((word_index[0] >= tag["tags_indexes"][0] and word_index[1] <= tag["tags_indexes"][1]) or (word_index[0] >= tag["tags_indexes"][0] and word_index[1]-1 <= tag["tags_indexes"][1] and word[-1] == ".")):
                # print("AEA!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                # print(word_index)
                # print(word_index[0], word_index[1])
                # print(teste[word_index[0]:word_index[1]])
                # print(tag["tags_indexes"])
                # print(tag["tag"])
                # print(tag["word"])

                tag_prefix = "B-"

                if result[-1]["count"] == count and result[-1]["tag"][2:] == tag["tag"]:
                    tag_prefix = "I-"

                if word[-1] == ".":
                    # result.append(f'Sentence: {count},{word[:-1]},{tag["tag"]}\n')
                    result.append({"count": count, "word": word[:-1], "tag": f'{tag_prefix}{tag["tag"]}'})
                    # result.append(f'Sentence: {count},{word[-1]},"O"\n')
                    result.append({"count": count, "word": word[-1], "tag": "O"})
                else:
                    # result.append(f'Sentence: {count},{word},{tag["tag"]}\n')
                    result.append({"count": count, "word": word, "tag": f'{tag_prefix}{tag["tag"]}'})
                found = True
                break
        if not found:
            # print("!!!!!!!!!!!!")
            if word[-1] == ".":
                # result.append(f'Sentence: {count},{word[:-1]},"O"\n')
                result.append({"count": count, "word": word[:-1], "tag": "O"})
                # result.append(f'Sentence: {count},{word[-1]},"O"\n')
                result.append({"count": count, "word": word[-1], "tag": "O"})
            else:
                # result.append(f'Sentence: {count},{word},"O"\n')
                result.append({"count": count, "word": word, "tag": "O"})
        # print(result[-2:])
    f1.close()
    f2.close()
    count = count + 1

result = [f'Sentence: {line["count"]},{line["word"]},{line["tag"]}\n' for line in result]

# print(result)
new_file = open("test.csv", "w", newline="\n", encoding="utf-8")
new_file.writelines(result)
new_file.close()
print(count)
# print(testeeee)
print(testeeee2)

# print(list(cwt.span_tokenize("3. Percocet , 5/325 , 1-2 tabs PO q4-6h prn pain .")))



Output hidden; open in https://colab.research.google.com to view.

In [20]:
full_text[769]

']'